In [ ]:
import warnings
warnings.filterwarnings('ignore')
from __future__ import division
%config InlineBackend.figure_format = 'retina'
%pylab inline

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import matplotlib.path as mplPath
import matplotlib.patches as patches

from matplotlib import rcParams
from matplotlib.font_manager import FontProperties
from matplotlib.patches import Polygon
from matplotlib.collections import PatchCollection
from mpl_toolkits.basemap import Basemap

import matplotlib.pyplot as plt
import matplotlib.cm


print matplotlib.matplotlib_fname()
from sys import platform


from scipy import optimize, special
from scipy.optimize import curve_fit
from sklearn.metrics import r2_score

reload(sys)
sys.setdefaultencoding('utf-8')

#from translate import translator
import goslate
from textblob import TextBlob
from textblob_de import TextBlobDE
import fileinput
import goslate

import shapefile
if platform == "linux" or platform == "linux2":
    prefix = "/home/massaro/"
elif platform == "darwin":
    prefix = "/Users/emanuele/"
    
font = {'family': 'sans-serif',
        'sans-serif' : 'Arial',
        'weight': 'normal',
        'size': 25}
plt.rc('font', **font)
from mpl_toolkits.axes_grid1 import make_axes_locatable
from mpl_toolkits.axes_grid1.inset_locator import inset_axes


powerlaw = lambda x, amp, index: amp * (x**index)

import urllib2

proxy_handler = urllib2.ProxyHandler({"http" : "http://proxy-domain.name:8080"})
proxy_opener = urllib2.build_opener(urllib2.HTTPHandler(proxy_handler),
                                    urllib2.HTTPSHandler(proxy_handler))

powerlaw = lambda x, amp, index: amp * (x**index)

import geopandas as gpd

In [ ]:
def loadData(folder, nameFile, sep):
    string = 'Dropbox (MIT)/HERUS_Lab/SwissDateset/'+ folder + '/' + nameFile 
    path = prefix + string
    tst = pd.read_csv(path, sep = sep)
    return tst

def translateColumns(col):
    translate = []
    translate.append(col[0])
    for i in col[1:]:
        b = TextBlob(i)
        print i
        A = b.translate(to='en')
        translate.append(str(A))
    return translate

powerlaw = lambda x, amp, index: amp * (x**index)

def plotScaling(xdata, ydata, init_vals, name):
    props = dict(boxstyle='round', facecolor='wheat', alpha=0.5)
    best_vals, covar = curve_fit(powerlaw, xdata, ydata, p0=init_vals)
    ypred = powerlaw(xdata, best_vals[0], best_vals[1])
    fig,ax = plt.subplots(figsize=(6,6))
    plt.plot(xdata, ydata, lw = 0, marker= "o")
    plt.plot(xdata, ypred)
    ax.set_xscale("log", nonposx='clip')
    ax.set_yscale("log", nonposy='clip')
    ax.set_xlabel('Inhabitants',size = 18)
    ax.set_ylabel(name,size = 18)
    string1 = r'$\beta$ = %.2f'%best_vals[1]
    string2 = r'$R^2$ = %.2f'%(r2_score(ydata, ypred))
    string3 = string1 + '\n'+ '\n' + string2
    ax.text(0.75, 0.1, string3, transform=ax.transAxes, fontsize=14, 
            verticalalignment='bottom', bbox=props)
    
def remove_non_ascii_2(text):
    return ''.join([i if ord(i) < 128 else '' for i in text])



In [ ]:
def plotMap(df, name, string, title, show):
    cmap = plt.get_cmap('gnuplot')
    norm = matplotlib.colors.Normalize(min(df[string]), max(df[string]))
    color_producer = matplotlib.cm.ScalarMappable(norm=norm, cmap=cmap)
    missingCities = []
    fig,ax = plt.subplots(figsize=(18,13))
    cout = 0
    tot = 0
    l = []
    for nshp in xrange(Nshp):
        if recs[cout][15] not in l:
            l.append(recs[cout][15])
            tot+=1
        cc = 'cyan'
        ec = 'black'
        nameCity = recs[cout][15]
        tmpCity = df[df[name] == nameCity]
        if len(tmpCity) > 0:
            cc = color_producer.to_rgba( tmpCity[string][tmpCity.index[0]]  )
        else:
            missingCities.append([nameCity, cout, recs[cout][17]])
            if recs[cout][17]=='LI':
                cc = 'white'
                ec = 'white'
            if recs[cout][17]=='DE':
                cc = 'white'
                ec = 'white'
            if recs[cout][17]=='IT':
                cc = 'dwhite'
                ec = 'white'
        cout+=1
        pts     = array(shapes[nshp].points)
        prt     = shapes[nshp].parts
        par     = list(prt) + [pts.shape[0]]
        bbPath = mplPath.Path(pts)
        ptchs   = []
        for pij in xrange(len(prt)):
            ptchs.append(Polygon(pts[par[pij]:par[pij+1]]))
            p = PatchCollection(ptchs,facecolor=cc, edgecolor=ec, linewidths = 1)
            ax.add_collection(p)
    ax.set_xlim(5.5, 11)
    ax.set_ylim(45.6,48.2)
    ax.set_xlabel('Latitude',size = 30)
    ax.set_ylabel('Longitude',size= 30)
    ax.set_title(title,size= 30)
    if show == 1:
        p.set(array= df[string].values, cmap='gnuplot')
        cbaxes = inset_axes(ax, width="50%", height="3%", loc=1) 
        plt.colorbar(p, cax=cbaxes, orientation='horizontal', format='%.0e')
    return missingCities

In [ ]:
### Locality Postal Code
path = prefix + 'Dropbox (MIT)/HERUS_Lab/SwissDateset/localityPostalCode/PLZO_CSV_LV03.csv'
zipCodes = pd.read_csv(path, sep= ';', encoding='utf-8')
col = ['cityName', 'zipCode', 'additionalClause', 'communityName', 'BFS-Nr', 'cantonAbbreviation', 'E', 'N']
zipCodes.columns = col
zipCodes['Name of commune'] = zipCodes['communityName'].apply(remove_non_ascii_2)

In [ ]:
zipCodes.head(5)

In [ ]:
## Municipale Boundaries
pathShape = prefix+'Dropbox (MIT)/HERUS_Lab/SwissDateset/boundaries/'
sf = shapefile.Reader(pathShape+"municipalities.shp")
recs    = sf.records()
shapes  = sf.shapes()
Nshp    = len(shapes)

In [ ]:
path = prefix+'Dropbox (MIT)/HERUS_Lab/SwissDateset/keyData/keyData2017.csv'
keyData2017 = pd.read_csv(path, sep=',', encoding='utf-8')
keyData2017['Name of commune'] = keyData2017['Name of commune'].apply(remove_non_ascii_2)
keyData2017['logResidents'] = log(keyData2017.Residents)

In [ ]:
keyData2017['OrginName'] = keyData2017['Name of commune']

In [ ]:
keyData2017['Name of commune'][1957] = "Chteau-d'Oex"
keyData2017['Name of commune'][2120] = "Crans-Montana"
keyData2017['Name of commune'][691]  = "Estavayer"
keyData2017['Name of commune'][2083] = "Val-d'Illiez"
keyData2017['Name of commune'][1584] = "Sant'Antonio"
keyData2017['Name of commune'][1973] = "L'Abbaye"
keyData2017['Name of commune'][1973] = "L'Isle"
keyData2017['Name of commune'][1668] = "Collina d'Oro"
keyData2017['Name of commune'][1583] =  "Sant'Antonino"
keyData2017['Name of commune'][1909] =  "L'Abergement"
keyData2017['Name of commune'][1670] =  "Comunanza Cadenazzo/Monteceneri"
keyData2017['Name of commune'][1655] =  "Comunanza Capriasca/Lugano"

In [ ]:
ms = plotMap(keyData2017, 'Name of commune', 'logResidents', 'Residents (log)', 1)

In [ ]:
ms = plotMap(keyData2017, 'Name of commune', 'Population density per km²', 'Density per Km2',1)

In [ ]:
tmp = keyData2017

tmp = tmp.merge(zipCodes, on='Name of commune')  # merge dataframe

ms = plotMap(tmp, 'Name of commune', 'zipCode', 'Postal Code', 0)

In [ ]:
zipCodes['Name of commune'][3014]

In [ ]:
ms

In [ ]:
pathS = '/home/massaro/Dropbox (MIT)/HERUS_Lab/SwissDateset/boundaries/'

In [ ]:
fig, ax = plt.subplots(figsize=(10,20))
m = Basemap(resolution='c', # c, l, i, h, f or None
            projection='merc',
            lat_0 = 40.5, lon_0=0.36,
            llcrnrlon=5, llcrnrlat= 45.5, urcrnrlon=11., urcrnrlat=48.5)
m.drawmapboundary(fill_color='#46bcec')
m.fillcontinents(color='#f2f2f2',lake_color='#46bcec')
m.drawcoastlines()
m.drawcountries(linewidth=3,color='black')
#m.readshapefile(pathS+"municipalities", 'areas')

In [ ]:
# Age Class
tst = loadData('ageClass/', 'tstst2.22011.csv', ";")
# Road Network
tmp = loadData('roadCalculation/', 'mantstst51.csv', ",")
tmp.columns = translateColumns(tmp.columns)  # translate columns
tst = tst.merge(tmp, on='Stadtname')  # merge dataframe

In [ ]:
#Unemployment
tmp = loadData('unemployment/', 'mantstst4.4.csv', ";")
tst = tst.merge(tmp, on='Stadtname')  # merge dataframe

In [ ]:
## Civil 
tmp = loadData('genderCivilStatus/', 'tstst2.3.csv', ";")
tmp.columns = translateColumns(tmp.columns)  # translate columns
tst = tst.merge(tmp, on='Stadtname')  # merge dataframe

In [ ]:
population = xdata = tst['Sex (women)'] + tst['Sex (Men)']

In [ ]:
init_vals = [0.01, 1.1]
name = 'arbeitslose_total'
ydata = tst[name]
plotScaling(xdata, ydata, init_vals, name)

In [ ]:
init_vals = [0.01, 1.1]
name = 'teilzeit_al_total'
ydata = tst[name]
plotScaling(xdata, ydata, init_vals, name)